# Tensorlow Obje Algılayıcısını Colab Üzerinde Eğitme

Başlamadan önce, gerekli klasörlerinizi drive'a upload etmeniz ve isteğe bağlı kernel'ı sıfırlamanız  gerekmektedir.



TODO: Resimleri rar dosyasında google'a al, csv, record gibi işlemleri buradan yap. Resmin değişmesi çok düşük ihtmaldir.

TODO: Yapıyı düzenle.

## Gereksinimlerin Yüklenmesi

Modellerin kullanılabilmesi için alttaki adımların **sırayla** yapılması gerekmektedir

### Tensorflow Modellerinin İndirilmesi

In [0]:
!git clone https://github.com/tensorflow/models.git

### Protobuf Derleyicisinin İndirilmesi

In [0]:
!apt-get install protobuf-compiler python-pil python-lxml

### Protobuf Dosyalarının Derlenmesi

Proto dosyalarının olduğu dizine `%cd` ile gelmezsen hata vermektedir.

In [0]:
%cd /content/models/research/
!protoc object_detection/protos/*.proto --python_out=.
%cd /content/

### Coco API kurulumu (İsteğe Bağlı)

In [0]:
!git clone https://github.com/cocodataset/cocoapi.git
%cd cocoapi/PythonAPI
!make
!cp -r pycocotools /content/models/research/
%cd ../..
!rm -rf cocoapi

###  Drive Dosyalarını Dosya Sistemine Bağlama

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

## Obje Algılama Modelinin Kullanılması



### Obje Algılama Modeli Üzerinde İşlemler

In [0]:
#@title Modeli Eğitme Aracı { display-mode: "form" }

MODEL_DIZINI = "content/gdrive/My Drive/Sim Model" #@param {type:"string"}
EGITIM_SCRIPTI = "train" #@param ["model_main", "train"]
MODELIN_CIKTI_DIZINI_YOLU = "content/gdrive/My Drive/Sim Model/training" #@param {type:"string"}
EGITIM_ADIM_SAYISI = 20000 #@param {type:"slider", min:100, max:20000, step:100}
YAPILANDIRMA_DOSYASI_YOLU = "content/gdrive/My Drive/Sim Model/configs/custom_ssdlite_mobilenet_v2_coco.config" #@param {type:"string"}
temiz_egitim = False #@param {type:"boolean"}
logdir = True #@param {type:"boolean"}
#@markdown <p><strong>Model Dizinine Erişim</strong></p><ul><li>Sol üst köşedeki <code>CO</code>simgesinin altındaki <code>&gt;</code>simgesine tıklayın</li><li><code>Files</code> sekmesine gelin</li><li>Scriptinizin olduğu klasörü bulun</li><li>Klasöre sağ tıklayıp <code>Copy Path</code> deyip, ilgili alana yapıştırın</li></ul>
#@markdown *Eğitim adım sayısı **sadece** 'model_main' dosyasında çalışmaktadır.*

# Ortam Değişkenlerini ayarlama
%env PYTHONPATH=/env/python:/content/models/research:/content/models/research/slim:/content/models/research/object_detection
# %env PYTHONPATH=$PYTHONPATH:/content/models/research:/content/models/research/slim:/content/models/research/object_detection

%cd '/{MODEL_DIZINI}'

if temiz_egitim:
  !rm -rf '/{MODELIN_CIKTI_DIZINI_YOLU}'
  print("Egitim dizini temizlendi.")

if EGITIM_SCRIPTI == "model_main":
  !python 'model_main.py' \
      --pipeline_config_path='/{YAPILANDIRMA_DOSYASI_YOLU}' \
      --model_dir='/{MODELIN_CIKTI_DIZINI_YOLU}' \
      --num_train_steps={EGITIM_ADIM_SAYISI} \
      --sample_1_of_n_eval_examples=1 \
      {'--alsologtostderr' if logdir else ''}
else:
  !python 'train.py' \
      --train_dir="/{MODELIN_CIKTI_DIZINI_YOLU}" \
      --pipeline_config_path="/{YAPILANDIRMA_DOSYASI_YOLU}" \
      {'--logtostderr' if logdir else ''}

%cd /content

In [0]:
#@title Sonuç Grafiğini Çıkarma Aracı { display-mode: "form" }
SCRIPT_DIZINI = "content/gdrive/My Drive/Sim Model" #@param {type:"string"}
YAPILANDIRMA_DOSYASI_YOLU = "configs/custom_ssdlite_mobilenet_v2_coco.config" #@param {type:"string"}
MODEL_DOSYASI_DIZINI = "training" #@param {type:"string"}
KONTROL_NOKTASI_ON_EKI = 8469 #@param {type:"integer"}
CIKTI_DIZINI = "inference_graph" #@param {type:"string"}
TEMIZ_CIKTI = True #@param {type:"boolean"}
# SONRAKI_EGITIME_HAZIRLA = False #@param {type:"boolean"}
#@markdown <p><strong>Script Yoluna Erişim</strong></p><ul><li>Sol üst köşedeki <code>CO</code>simgesinin altındaki <code>&gt;</code>simgesine tıklayın</li><li><code>Files</code> sekmesine gelin</li><li>Scriptinizin olduğu klasörü bulun</li><li>Klasöre sağ tıklayıp <code>Copy Path</code> deyip, ilgili alana yapıştırın</li></ul>
# #@markdown <p><strong>Bayrak Tanımlamaları</strong></p><ul><li><code>TEMIZ_CIKTI</code> Eskisini siler</li><li><code>SONRAKI_EGITIME_HAZIRLA</code> Modeli ek dosya yerine pre_trained_model üzerine yazar</li></ul>

# Ortam Değişkenlerini ayarlama
%env PYTHONPATH=/env/python:/content/models/research:/content/models/research/slim:/content/models/research/object_detection
# %env PYTHONPATH=$PYTHONPATH:/content/models/research:/content/models/research/slim:/content/models/research/object_detection

%cd "/{SCRIPT_DIZINI}"

if TEMIZ_CIKTI:
  !rm -rf "{CIKTI_DIZINI}"

!python export_inference_graph.py --input_type image_tensor --pipeline_config_path {YAPILANDIRMA_DOSYASI_YOLU} --trained_checkpoint_prefix {MODEL_DOSYASI_DIZINI}/model.ckpt-{KONTROL_NOKTASI_ON_EKI} --output_directory {CIKTI_DIZINI}

# if SONRAKI_EGITIME_HAZIRLA:
#   !rm -rf "pre_trained_model"
#   !mv "{CIKTI_DIZINI}" "pre_trained_model"

%cd /content

## Test Aşamasındaki İşlemler

###  Çalışma Ortamının Hazırlanması

Yapılandırma dosyasında belirtilmesi lazım recordların yolunun.

In [0]:
#@title Eğitilecek Modeli Tensorflow Zoo'dan İndirme ve Konumlandırma { display-mode: "form" }
MODEL_URL = "http://download.tensorflow.org/models/object_detection/ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz" #@param {type:"string"}
KONUM_YOLU = "content/gdrive/My Drive/Sim Model/pre_trained_model" #@param {type:"string"}
TEMIZ_TASIMA = True #@param {type:"boolean"}

%cd "/content"
!wget "{MODEL_URL}" # Modeli indirme

model_name = MODEL_URL.split('/').pop()
!tar xvzf "/content/{model_name}" # Zip'den çıkarma


!rm -rf "/content/{model_name}" # Zip'i silme
!rm -rf "/{KONUM_YOLU}" if TEMIZ_TASIMA else pass # Eskisini silme

model_bare_name = model_name.split('.tar.gz')[0]
!mv "/content/{model_bare_name}" "/{KONUM_YOLU}" # Taşıma
print("Kod tamamlanmıştır. (YEmreAk)")


In [0]:
#@title Model Dosyaları İndirici { display-mode: "form" }
#@markdown > ***Yüksek boyutlu dosyalarda hata vermekte***
LABEL_MAP_PBTXT_URL = "https://drive.google.com/open?id=1yma4e7GMmxdKNsD6uKTnC1s1rAem3vOY"  # @param {type:"string"}
TRAIN_RECORD_URL = "https://drive.google.com/uc?export=download&confirm=SXjU&id=1jFE1adOYpKfxRskPnUMR3umvDcY2Nz_y"  # @param {type:"string"}
TEST_RECORD_URL = "https://drive.google.com/uc?export=download&confirm=oqvZ&id=1nl7a5-Fhw8q1OMYfEUNjeAfkH-Z1Y4Rz"  # @param {type:"string"}
CONFIG_FILE_URL = "https://drive.google.com/open?id=11USMBxsejlSJGu7RND9AjFMConp4YadJ"  # @param {type:"string"}
PRE_TRAINED_MODEL_URL = "http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz"  # @param {type:"string"}
TEMIZ_TASIMA = True #@param {type:"boolean"}

def fix_drive_url(url):
  if "drive.google.com/open?" in url:
    return url.replace(r"open?", r"uc?export=download&")

!wget -O label_map.pbtxt "{fix_drive_url(LABEL_MAP_PBTXT_URL)}"
!wget -O train.record "{fix_drive_url(TRAIN_RECORD_URL)}"
!wget -O test.record "{fix_drive_url(TEST_RECORD_URL)}"
!wget -O model_pipeline.config "{fix_drive_url(CONFIG_FILE_URL)}"

!wget "{PRE_TRAINED_MODEL_URL}" # Modeli indirme

model_name = PRE_TRAINED_MODEL_URL.split('/').pop()
!tar xvzf "/content/{model_name}" # Zip'den çıkarma

!rm -rf "/content/{model_name}" # Zip'i silme
!rm -rf "/content/pre_trained_model" if TEMIZ_TASIMA else pass # Eskisini silme

model_bare_name = model_name.split('.')[0]
!mv "/content/{model_bare_name}" "/content/pre_trained_model" # Taşıma

In [0]:
#@title Temel Dosya İşlemleri { display-mode: "form" }
DOSYA_YOLU = "content/custom_ssdlite_mobilenet_v2_coco.config" #@param {type:"string"}
ISLEM = "tasi" #@param ["oku", "tasi", "kopyala", "sil"]
TASINACAK_YOL = "content/gdrive/My Drive/Sim Model/configs/custom_ssdlite_mobilenet_v2_coco.config" #@param {type:"string"}

if ISLEM is "oku":
  !cat "/{DOSYA_YOLU}"
elif ISLEM is "tasi":
  !mv "/{DOSYA_YOLU}" "/{TASINACAK_YOL}"
elif ISLEM is "kopyala":
  !cp "/{DOSYA_YOLU}" "/{TASINACAK_YOL}"
elif ISLEM is "sil":
  !rm -rf "/{DOSYA_YOLU}"
  
print("Kod tamamlanmıştır. ~YEmreAk")


In [0]:
#@title Hatalı Resim Bulucu
SCRIPT_YOLU = "content/check_images.py" #@param {type:"string"}
RESIMLERIN_YOLU = "" #@param {type:"string"}


!unrar e "/content/images.rar" "/content/images" # Zip'den çıkarma
# !python "/{SCRIPT_YOLU}" -i 


## Kernel'ı sıfırlama (İsteğe Bağlı)

In [0]:
!kill -9 -1

## Geliştirici Alanı

**Bu alandaki bilgiler kendim için alınmış notları barındırır.**

Sorun durumunda model dizinine drive üzerinden girip `pre_trained_model` klasörünü indirin

In [0]:
#@title Sonuç İndirme Arayüzü
INDIRILECEK_DIZININ_YOLU = "content/gdrive/My Drive/Projects/Model/pre_trained_model" #@param {type:"string"}

from google.colab import files

# Dizin adını alma
folder_name = INDIRILECEK_DIZININ_YOLU.split('/').pop()

# Gerekli dosyaları oluşturma
!cp -r "/{INDIRILECEK_DIZININ_YOLU}" "/content"
!zip -r '{folder_name}.zip'  "{folder_name}"

# İndirme işlemini başlatma
files.download(f'{folder_name}.zip')

# Geçici dosyaları temizleme
!rm -rf '{folder_name}.zip'
!rm -rf '{folder_name}'


In [0]:
# !ls
# !cat "/content/gdrive/My Drive/Sim Model/configs/ssd_mobilenet_v2_coco.config"
# !rm -f /content/gdrive/My Drive/Sim Model/train.py
# !mv "/content/train.py" "/content/gdrive/My Drive/Sim Model/train.py"

In [0]:
# !wget "http://download.tensorflow.org/models/object_detection/ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz"
# !tar xvzf "/content/ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz"
# !rm -rf "/content/ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz"
!mv "/content/ssd_mobilenet_v2_coco.config" "/content/gdrive/My Drive/Sim Model/configs/ssd_mobilenet_v2_coco.config"
!cat "/content/gdrive/My Drive/Sim Model/configs/ssd_mobilenet_v2_coco.config"

### Obje Algılama Modelini Eğitme (Kullanım Dışı)

Modelin eğitimi `train.py` script dosyası ile yapılamktadır.

### Tensorflow 2.0 için (Deneysel Kısım)

### Modeli TensorBoard Üzerinde İzleme (Deneysel)

TensorBoard arayüzü görsellik ve anlaşılabilirlik açısından oldukça faydalıdır.


> **Bilgi sahibi değilseniz bu adımı ve alt adımlarını uygulamayın**
>
> **AttributeError: module 'tensorflow' has no attribute 'contrib hatasına sebep oluyor is VM'i sıfırlayın** 
```py
kill -9 -1
```

#### Kernel Sıfırlama

In [0]:
!kill -9 -1

#### Gerekli Eklentinin Kurulumu

In [0]:
!pip install -q tf-nightly-2.0-preview
# Load the TensorBoard notebook extension
%load_ext tensorboard.notebook

#### Eklenti Yardımıyla TensorBoard Arayüzünü Açma

In [0]:
%tensorboard --logdir "/content/gdrive/My Drive/Projects/Yazılım Ekibi | Otobil/Yunus Emre/Model/training/"

### Hata Notları

- [Allocation of X exceeds 10% of system memory](https://github.com/tensorflow/tensorflow/issues/18736#issuecomment-385976699)
- [Allocation of X exceeds 10% of system memory](https://github.com/tensorflow/tensorflow/issues/18736#issuecomment-388709455)